In [ ]:
#!pip install langchain-community
#!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.23
    Uninstalling langsmith-0.3.23:
      Successfully uninstalled langsmith-0.3.23
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  A

In [2]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document
from cuml.cluster import KMeans
import numpy as np
import torch
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

REPEAT = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading data...")
df = pd.concat([pd.read_csv('/kaggle/input/ag-news-classification-dataset/train.csv'), 
                pd.read_csv('/kaggle/input/ag-news-classification-dataset/test.csv')])
classmap = {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Science/Technology'}
df['Class Index'] = df['Class Index'].map(classmap)

rows = df[['Title', 'Description', 'Class Index']].fillna("")

Loading data...


In [3]:
print("Initializing embedding model...")
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", 
                                               model_kwargs={"device": device})

print("Creating augmented text chunks...")
texts = []
for _, row in tqdm(rows.iterrows(), total=len(rows), desc="Formatting text"):
    article_text = f"Title: {row['Title']}\nDescription: {row['Description']}\nCategory: {row['Class Index']}"
    augmented_text = "\n".join([article_text] * REPEAT)
    texts.append(augmented_text)

print("Generating embeddings for augmented texts...")
batch_size = 512
embeddings = []
documents = []
for i in tqdm(range(0, len(texts), batch_size), desc="Embedding texts"):
    batch = texts[i:i+batch_size]
    batch_embeddings = embedding_model.embed_documents(batch)
    embeddings.extend(batch_embeddings)
    batch_docs = [Document(page_content=text, metadata={"index": i+j}) 
                  for j, text in enumerate(batch)]
    documents.extend(batch_docs)
embeddings = np.array(embeddings, dtype=np.float32)

print("Creating FAISS index...")
vectorstore = FAISS.from_documents(documents, embedding_model)

print("Saving FAISS index...")
vectorstore.save_local("faiss_index")

print("Embedding generation complete.")

Initializing embedding model...


2025-06-13 14:49:05.475865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749826145.657859      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749826145.711740      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Creating augmented text chunks...


Formatting text: 100%|██████████| 127600/127600 [00:05<00:00, 24977.50it/s]


Generating embeddings for augmented texts...


Embedding texts: 100%|██████████| 250/250 [05:19<00:00,  1.28s/it]


Creating FAISS index...
Saving FAISS index...
Embedding generation complete.
